In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.stats import chi2_contingency, f_oneway
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [125]:
print(sys.version)

3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:52:10) 
[Clang 14.0.6 ]


In [126]:

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import mutual_info_classif


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [127]:
df_train = pd.read_csv("GUIDE_Train.csv", nrows=10000)
df_test = pd.read_csv("GUIDE_Test.csv", nrows=10000)

In [128]:
df_train.head(5)

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,...,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,180388628218,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,455266534868,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,1056561957389,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,1279900258736,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,214748368522,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [129]:
df_train = df_train.drop(columns = ["Id","OrgId","IncidentId","AlertId","DetectorId","DeviceId"])
df_test = df_test.drop(columns = ["Id","OrgId","IncidentId","AlertId","DetectorId","DeviceId","Usage"])

In [130]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           10000 non-null  object 
 1   AlertTitle          10000 non-null  int64  
 2   Category            10000 non-null  object 
 3   MitreTechniques     4201 non-null   object 
 4   IncidentGrade       9947 non-null   object 
 5   ActionGrouped       59 non-null     object 
 6   ActionGranular      59 non-null     object 
 7   EntityType          10000 non-null  object 
 8   EvidenceRole        10000 non-null  object 
 9   Sha256              10000 non-null  int64  
 10  IpAddress           10000 non-null  int64  
 11  Url                 10000 non-null  int64  
 12  AccountSid          10000 non-null  int64  
 13  AccountUpn          10000 non-null  int64  
 14  AccountObjectId     10000 non-null  int64  
 15  AccountName         10000 non-null  int64  
 16  Devic

In [131]:

#missing_values = df_train.isnull().sum()
#missing_percentages = 100 * missing_values / len(df_train)

# Display columns with missing values
#missing_data = pd.concat([missing_values, missing_percentages], axis=1, keys=['Total', 'Percent'])
#print(missing_data[missing_data['Total'] > 0].sort_values('Percent', ascending=False))


In [132]:
def drop_column_with_over_50_percent_missing(df):
    missing_values = df.isnull().sum()
    missing_percentages = 100 * missing_values / len(df)

    # Display columns with missing values
    missing_data = pd.concat([missing_values, missing_percentages], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Total'] > 0].sort_values('Percent', ascending=False))
    
    
    list = missing_data[missing_data.Percent > 50].index.tolist()
    df_drop = df.drop(columns = list)
    return df_drop


df_train_drop = drop_column_with_over_50_percent_missing(df_train)
df_test_drop = drop_column_with_over_50_percent_missing(df_test)

                   Total  Percent
ResourceType        9992    99.92
ActionGrouped       9941    99.41
ActionGranular      9941    99.41
ThreatFamily        9923    99.23
EmailClusterId      9881    98.81
AntispamDirection   9823    98.23
Roles               9789    97.89
SuspicionLevel      8462    84.62
LastVerdict         7600    76.00
MitreTechniques     5799    57.99
IncidentGrade         53     0.53
                   Total  Percent
ActionGrouped       9996    99.96
ActionGranular      9996    99.96
ResourceType        9992    99.92
ThreatFamily        9928    99.28
EmailClusterId      9905    99.05
AntispamDirection   9801    98.01
Roles               9749    97.49
SuspicionLevel      8454    84.54
LastVerdict         7609    76.09
MitreTechniques     5535    55.35


In [133]:
df_train_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Timestamp           10000 non-null  object
 1   AlertTitle          10000 non-null  int64 
 2   Category            10000 non-null  object
 3   IncidentGrade       9947 non-null   object
 4   EntityType          10000 non-null  object
 5   EvidenceRole        10000 non-null  object
 6   Sha256              10000 non-null  int64 
 7   IpAddress           10000 non-null  int64 
 8   Url                 10000 non-null  int64 
 9   AccountSid          10000 non-null  int64 
 10  AccountUpn          10000 non-null  int64 
 11  AccountObjectId     10000 non-null  int64 
 12  AccountName         10000 non-null  int64 
 13  DeviceName          10000 non-null  int64 
 14  NetworkMessageId    10000 non-null  int64 
 15  RegistryKey         10000 non-null  int64 
 16  RegistryValueName   100

In [134]:
def fill_missing_value(df, column):
    df[column].fillna(df[column].mode()[0], inplace = True)

fill_missing_value(df_train_drop, 'IncidentGrade')
fill_missing_value(df_test_drop, 'IncidentGrade')

/var/folders/dt/7vnpb4hn01d19x7dfz27zs5h0000gn/T/ipykernel_88547/2763356033.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace = True)


In [135]:
numerical_columns = df_train_drop.select_dtypes(include=['int64']).columns

In [136]:
cat_columns = df_train_drop.select_dtypes(include=['object']).columns

In [137]:
df_train_drop[numerical_columns].nunique().sort_values(ascending=False)

AccountUpn            3386
AccountName           2323
AccountSid            2211
AlertTitle            2205
AccountObjectId       2200
IpAddress             1879
NetworkMessageId      1193
FileName               706
DeviceName             686
Url                    603
Sha256                 512
FolderPath             465
City                   196
State                  136
CountryCode             68
ApplicationName         23
ApplicationId           21
ResourceIdName          12
RegistryKey             10
OSVersion                6
RegistryValueName        5
RegistryValueData        4
OAuthApplicationId       3
OSFamily                 3
dtype: int64

In [138]:
df_train_drop[cat_columns].nunique().sort_values(ascending=False)

Timestamp        9580
EntityType         20
Category           17
IncidentGrade       3
EvidenceRole        2
dtype: int64

In [139]:
df_train_drop[numerical_columns]

,AlertTitle,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,...,ApplicationName,OAuthApplicationId,FileName,FolderPath,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City
0,6,138268,27,160396,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,31,6,3
1,43,138268,360606,160396,22406,23032,22795,24887,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
2,298,138268,360606,68652,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
3,2,138268,360606,13,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
4,74,138268,360606,160396,449,592,440,479,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,138268,360606,160396,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
9996,1,138268,360606,160396,45848,53590,43569,41163,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
9997,29882,138268,360606,160396,2450,3209,2424,2567,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
9998,12372,138268,360606,160396,441377,673934,425863,453297,153085,529644,...,8,881,289573,117668,3586,5,66,242,1445,10630


In [140]:
df_train_drop.columns.unique()

Index(['Timestamp', 'AlertTitle', 'Category', 'IncidentGrade', 'EntityType',
       'EvidenceRole', 'Sha256', 'IpAddress', 'Url', 'AccountSid',
       'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName',
       'NetworkMessageId', 'RegistryKey', 'RegistryValueName',
       'RegistryValueData', 'ApplicationId', 'ApplicationName',
       'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName',
       'OSFamily', 'OSVersion', 'CountryCode', 'State', 'City'],
      dtype='object')

In [141]:
def timestamp_transform(df):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['DayOfWeek'] = df['Timestamp'].dt.day_name() 
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

timestamp_transform(df_train_drop)
timestamp_transform(df_test_drop)

In [142]:
import pandas as pd
from scipy.stats import chi2_contingency, f_oneway
from sklearn.feature_selection import mutual_info_classif

# Significance threshold
alpha = 0.05

# Chi-Square Test for Categorical Features
def chi_square_test(feature, target):
    contingency_table = pd.crosstab(df_train_drop[feature], df_train_drop[target])
    chi2, p, dof, ex = chi2_contingency(contingency_table)
    return p

# ANOVA Test for Numerical Features
def anova_test(feature, target):
    groups = df_train_drop.groupby(target)[feature].apply(list)
    f_stat, p_value = f_oneway(*groups)
    return p_value

# Mutual Information for Numerical Features
def mutual_information(feature, target):
    mi = mutual_info_classif(df_train_drop[[feature]], df_train_drop[target])
    return mi[0]

chi_square_results = {}
anova_results = {}
mutual_info_results = {}

insignificant_features = []

for feature in df_train_drop.columns:
    if feature != 'IncidentGrade':
        if isinstance(df_train_drop[feature].dtype, pd.CategoricalDtype) or pd.api.types.is_object_dtype(df_train_drop[feature]):
            p_value = chi_square_test(feature, 'IncidentGrade')
            chi_square_results[feature] = p_value
            if p_value > alpha:
                insignificant_features.append((feature, 'Chi-Square', p_value))
        elif pd.api.types.is_numeric_dtype(df_train_drop[feature]):
            p_value = anova_test(feature, 'IncidentGrade')
            mi_score = mutual_information(feature, 'IncidentGrade')
            anova_results[feature] = p_value
            mutual_info_results[feature] = mi_score
            if p_value > alpha:
                insignificant_features.append((feature, 'ANOVA', p_value))

# Sort results
sorted_chi_square_results = sorted(chi_square_results.items(), key=lambda item: item[1], reverse=True)
sorted_anova_results = sorted(anova_results.items(), key=lambda item: item[1], reverse=True)
sorted_mi_results = sorted(mutual_info_results.items(), key=lambda item: item[1], reverse=True)

# Print insignificant features
if insignificant_features:
    print("Insignificant Features:")
    for feature, test, p_value in insignificant_features:
        print(f"Feature: {feature}, Test: {test}, P-Value: {p_value}, mutual info: {mi_score}")
else:
    print("All features are significant at the alpha level of", alpha)

# Optionally, print or return the sorted significant results
# print(sorted_chi_square_results, sorted_anova_results, sorted_mi_results)


Insignificant Features:
Feature: RegistryValueName, Test: ANOVA, P-Value: 0.3316949265418009, mutual info: 0
Feature: RegistryValueData, Test: ANOVA, P-Value: 0.33124938314252445, mutual info: 0


In [143]:
df_train_significant = df_train_drop.drop(columns = ["RegistryValueName","RegistryValueData"])
df_test_significant = df_test_drop.drop(columns = ["RegistryValueName","RegistryValueData"])

In [144]:
df_train_significant.describe()

,AlertTitle,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,...,OAuthApplicationId,FileName,FolderPath,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City,IsWeekend
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,3164.179100,129100.140100,285580.162600,150216.553800,352084.509800,463508.998600,340213.367800,356221.925200,143331.718700,480467.433600,...,880.824000,263596.523000,108249.389000,3582.883900,4.885200,64.484700,224.151900,1353.582200,9952.959800,0.175300
std,11894.681561,33313.245639,141912.774616,37804.399327,166574.325045,290380.115552,160236.388792,174785.603137,35807.555104,141219.984108,...,12.444465,79792.593847,31253.129395,100.103536,0.748517,9.877941,62.023338,347.523692,2580.017748,0.380243
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,138268.000000,360606.000000,160396.000000,441377.000000,90571.500000,425863.000000,441519.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
50%,11.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
75%,208.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
max,109424.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,1.000000


Encoding

In [145]:
df_train_significant.nunique().sort_values(ascending=False)

Timestamp             9580
AccountUpn            3386
AccountName           2323
AccountSid            2211
AlertTitle            2205
AccountObjectId       2200
IpAddress             1879
NetworkMessageId      1193
FileName               706
DeviceName             686
Url                    603
Sha256                 512
FolderPath             465
City                   196
State                  136
CountryCode             68
ApplicationName         23
ApplicationId           21
EntityType              20
Category                17
ResourceIdName          12
RegistryKey             10
DayOfWeek                7
OSVersion                6
OAuthApplicationId       3
OSFamily                 3
IncidentGrade            3
EvidenceRole             2
IsWeekend                2
dtype: int64

In [146]:
df_train_X = df_train_significant.drop(columns=["IncidentGrade"])
df_train_Y = df_train_significant["IncidentGrade"]
df_test_X = df_test_significant.drop(columns=["IncidentGrade"])
df_test_Y = df_test_significant["IncidentGrade"]

In [147]:
def target_encode(df, n_bins):
    """
    Preprocesses the data by performing target encoding, frequency encoding, 
    and hashing for high cardinality columns.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame to preprocess.
    - low_cardinality_columns (list): List of columns with low cardinality for frequency encoding.
    - high_cardinality_columns (list): List of columns with high cardinality for hashing.
    - target_column (str): The name of the target variable for target encoding.
    - n_bins (int, optional): Number of bins for hashing high cardinality columns. Default is 100.
    
    Returns:
    - pd.DataFrame: The preprocessed DataFrame.
    """
    
    low_cardinality_columns = df.drop(columns = ["Timestamp"]).nunique()[df.nunique() <= 10].index
    high_cardinality_columns = df.drop(columns = ["Timestamp"]).nunique()[df.nunique() > 10].index

    # Copy the DataFrame to avoid altering the original data
    df_encoded = df.copy()
    
    # Drop the 'Timestamp' column
    if 'Timestamp' in df_encoded.columns:
        df_encoded = df_encoded.drop('Timestamp', axis=1)
    
    # Target encoding
    encoder = TargetEncoder()
    
    # Perform frequency encoding for low cardinality columns
    for column in low_cardinality_columns:
        df_encoded[column] = df_encoded[column].map(df_encoded[column].value_counts())
    
    # Perform hashing for high cardinality columns
    for column in high_cardinality_columns:
        df_encoded[column] = df_encoded[column].apply(lambda x: np.abs(hash(str(x))) % n_bins)
    
    return df_encoded

In [148]:
X_train_encoded = target_encode(df_train_X, 100)
X_test_encoded = target_encode(df_test_X, 100)

In [149]:
X_test_encoded.nunique().sort_values(ascending=False)

AlertTitle            100
AccountSid            100
FileName              100
NetworkMessageId      100
AccountObjectId       100
AccountUpn            100
AccountName           100
IpAddress             100
Url                    99
Sha256                 99
DeviceName             99
FolderPath             97
City                   86
State                  76
CountryCode            47
ApplicationName        22
EntityType             19
ApplicationId          16
Category               14
DayOfWeek               7
OSVersion               6
RegistryKey             4
OSFamily                4
ResourceIdName          3
OAuthApplicationId      3
EvidenceRole            2
IsWeekend               2
dtype: int64

In [150]:
# Scaling

def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled



X_train_scaled, X_test_scaled = scale_data(X_train_encoded, X_test_encoded)

# Model Selection

In [151]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, df_train_Y)
y_pred_rf = rf_model.predict(X_test_scaled)

print("Random Forest Classifier Report:")
print(classification_report(df_test_Y, y_pred_rf))

Random Forest Classifier Report:
                precision    recall  f1-score   support

BenignPositive       0.63      0.80      0.70      4192
 FalsePositive       0.58      0.31      0.41      2167
  TruePositive       0.72      0.68      0.70      3641

      accuracy                           0.65     10000
     macro avg       0.64      0.60      0.60     10000
  weighted avg       0.65      0.65      0.64     10000



In [152]:
# Gradient Boost

from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_scaled, df_train_Y)
y_pred_gb = gb_model.predict(X_test_scaled)

print("Gradient Boosting Classifier Report:")
print(classification_report(df_test_Y, y_pred_gb))

Gradient Boosting Classifier Report:
                precision    recall  f1-score   support

BenignPositive       0.56      0.98      0.71      4192
 FalsePositive       0.79      0.04      0.07      2167
  TruePositive       0.82      0.56      0.67      3641

      accuracy                           0.62     10000
     macro avg       0.72      0.53      0.48     10000
  weighted avg       0.70      0.62      0.56     10000



In [153]:
# XGBoost

from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=42)

le = LabelEncoder()
label_train_y = le.fit_transform(df_train_Y)

# BenignPositive = 0
# FalsePositive = 1
# TruePositive = 2

xgb_model.fit(X_train_scaled, label_train_y)
y_pred_xgb = xgb_model.predict(X_test_scaled)
label_test_y = le.fit_transform(y_pred_xgb)
print("XGBoost Classifier Report:")
print(classification_report(label_train_y, y_pred_xgb))

XGBoost Classifier Report:
              precision    recall  f1-score   support

           0       0.44      0.74      0.55      4444
           1       0.23      0.02      0.04      2110
           2       0.35      0.23      0.28      3446

    accuracy                           0.41     10000
   macro avg       0.34      0.33      0.29     10000
weighted avg       0.36      0.41      0.35     10000



In [154]:
# LightGBM

from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train_scaled, df_train_Y)
y_pred_lgbm = lgbm_model.predict(X_test_scaled)

print("LightGBM Classifier Report:")
print(classification_report(df_test_Y, y_pred_lgbm))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002025 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1012
[LightGBM] [Info] Number of data points in the train set: 10000, number of used features: 25
[LightGBM] [Info] Start training from score -0.811030
[LightGBM] [Info] Start training from score -1.555897
[LightGBM] [Info] Start training from score -1.065371
LightGBM Classifier Report:
                precision    recall  f1-score   support

BenignPositive       0.63      0.89      0.74      4192
 FalsePositive       0.69      0.31      0.43      2167
  TruePositive       0.78      0.66      0.72      3641

      accuracy                           0.68     10000
     macro avg       0.70      0.62      0.63     10000
  weighted avg       0.70      0.68      0.67     10000



In [155]:
#CatBoost

from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier(random_state=42, verbose=0)
catboost_model.fit(X_train_scaled, df_train_Y)
y_pred_catboost = catboost_model.predict(X_test_scaled)

print("CatBoost Classifier Report:")
print(classification_report(df_test_Y, y_pred_catboost))

CatBoost Classifier Report:
                precision    recall  f1-score   support

BenignPositive       0.62      0.89      0.73      4192
 FalsePositive       0.67      0.29      0.40      2167
  TruePositive       0.77      0.65      0.71      3641

      accuracy                           0.67     10000
     macro avg       0.69      0.61      0.61     10000
  weighted avg       0.69      0.67      0.65     10000



In [156]:
def model_comparison(X_train_scaled, X_test_scaled, df_train_Y, df_test_Y):
    results = {}

    # Random Forest
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train_scaled, df_train_Y)
    y_pred_rf = rf_model.predict(X_test_scaled)

    rf_report = classification_report(df_test_Y, y_pred_rf, output_dict=True)
    results['RandomForest'] = rf_report['macro avg']
    results['RandomForest'].update(rf_report['weighted avg'])
    results['RandomForest']['accuracy'] = rf_report['accuracy']

    #print("Random Forest Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_rf))

    # Gradient Boost
    gb_model = GradientBoostingClassifier(random_state=42)
    gb_model.fit(X_train_scaled, df_train_Y)
    y_pred_gb = gb_model.predict(X_test_scaled)

    gb_report = classification_report(df_test_Y, y_pred_gb, output_dict=True)
    results['GradientBoost'] = gb_report['macro avg']
    results['GradientBoost'].update(gb_report['weighted avg'])
    results['GradientBoost']['accuracy'] = gb_report['accuracy']

    #print("Gradient Boosting Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_gb))

    # XGBoost
    xgb_model = XGBClassifier(random_state=42)
    le = LabelEncoder()
    label_train_y = le.fit_transform(df_train_Y)
    label_test_y = le.transform(df_test_Y)

    xgb_model.fit(X_train_scaled, label_train_y)
    y_pred_xgb = xgb_model.predict(X_test_scaled)

    xgb_report = classification_report(label_test_y, y_pred_xgb, output_dict=True)
    results['XGBoost'] = xgb_report['macro avg']
    results['XGBoost'].update(xgb_report['weighted avg'])
    results['XGBoost']['accuracy'] = xgb_report['accuracy']

    #print("XGBoost Classifier Report:")
    #print(classification_report(label_test_y, y_pred_xgb))

    # LightGBM
    lgbm_model = LGBMClassifier(random_state=42)
    lgbm_model.fit(X_train_scaled, df_train_Y)
    y_pred_lgbm = lgbm_model.predict(X_test_scaled)

    lgbm_report = classification_report(df_test_Y, y_pred_lgbm, output_dict=True)
    results['LightGBM'] = lgbm_report['macro avg']
    results['LightGBM'].update(lgbm_report['weighted avg'])
    results['LightGBM']['accuracy'] = lgbm_report['accuracy']

    #print("LightGBM Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_lgbm))

    # CatBoost
    catboost_model = CatBoostClassifier(random_state=42, verbose=0)
    catboost_model.fit(X_train_scaled, df_train_Y)
    y_pred_catboost = catboost_model.predict(X_test_scaled)

    catboost_report = classification_report(df_test_Y, y_pred_catboost, output_dict=True)
    results['CatBoost'] = catboost_report['macro avg']
    results['CatBoost'].update(catboost_report['weighted avg'])
    results['CatBoost']['accuracy'] = catboost_report['accuracy']

    #print("CatBoost Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_catboost))

    # Convert the results dictionary into a DataFrame
    results_df = pd.DataFrame(results)

    # Transpose the DataFrame for better readability
    results_df = results_df.T

    # Display the combined DataFrame
    return results_df

In [157]:
model_comparison(X_train_scaled, X_test_scaled, df_train_Y, df_test_Y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000828 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1012
[LightGBM] [Info] Number of data points in the train set: 10000, number of used features: 25
[LightGBM] [Info] Start training from score -0.811030
[LightGBM] [Info] Start training from score -1.555897
[LightGBM] [Info] Start training from score -1.065371


,precision,recall,f1-score,support,accuracy
RandomForest,0.649829,0.6525,0.637450,10000.0,0.6525
GradientBoost,0.702448,0.6244,0.556020,10000.0,0.6244
XGBoost,0.701112,0.6227,0.562034,10000.0,0.6227
LightGBM,0.698953,0.6840,0.665072,10000.0,0.6840
CatBoost,0.687265,0.6726,0.651657,10000.0,0.6726


In [158]:



# Assume X_train_scaled and df_train_Y are already defined
# X_train_scaled: Scaled features for training
# df_train_Y: Target variable for training

# Initialize K-Fold cross-validation with 5 folds


def cross_validation(X_train_scaled, df_train_Y, kf):
    # Initialize a dictionary to hold the cross-validation results
    cv_results = {}

    # Random Forest
    rf_model = RandomForestClassifier(random_state=42)
    rf_scores = cross_val_score(rf_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['RandomForest'] = rf_scores
    #print(f"Random Forest CV Accuracy: {rf_scores.mean():.4f} (+/- {rf_scores.std():.4f})")

    # Gradient Boosting
    gb_model = GradientBoostingClassifier(random_state=42)
    gb_scores = cross_val_score(gb_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['GradientBoost'] = gb_scores
    #print(f"Gradient Boosting CV Accuracy: {gb_scores.mean():.4f} (+/- {gb_scores.std():.4f})")

    # XGBoost
    xgb_model = XGBClassifier(random_state=42)
    le = LabelEncoder()
    label_train_y = le.fit_transform(df_train_Y)

    xgb_scores = cross_val_score(xgb_model, X_train_scaled, label_train_y, cv=kf, scoring='accuracy')
    cv_results['XGBoost'] = xgb_scores
    #print(f"XGBoost CV Accuracy: {xgb_scores.mean():.4f} (+/- {xgb_scores.std():.4f})")

    # LightGBM
    lgbm_model = LGBMClassifier(random_state=42)
    lgbm_scores = cross_val_score(lgbm_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['LightGBM'] = lgbm_scores
    #print(f"LightGBM CV Accuracy: {lgbm_scores.mean():.4f} (+/- {lgbm_scores.std():.4f})")

    # CatBoost
    catboost_model = CatBoostClassifier(random_state=42, verbose=0)
    catboost_scores = cross_val_score(catboost_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['CatBoost'] = catboost_scores
    #print(f"CatBoost CV Accuracy: {catboost_scores.mean():.4f} (+/- {catboost_scores.std():.4f})")

    # Convert the cross-validation results into a DataFrame for easier comparison
    cv_results_df = pd.DataFrame(cv_results)

    # Transpose the DataFrame for better readability
    cv_results_df = cv_results_df.T

    # Add a column for the mean accuracy and standard deviation
    cv_results_df['Mean Accuracy'] = cv_results_df.mean(axis=1)
    cv_results_df['Std Dev'] = cv_results_df.std(axis=1)

    # Display the combined cross-validation results
    #print(cv_results_df)
    return cv_results_df

In [159]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_validation(X_train_scaled, df_train_Y, kf)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000566 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 24
[LightGBM] [Info] Start training from score -0.807717
[LightGBM] [Info] Start training from score -1.566618
[LightGBM] [Info] Start training from score -1.063124
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000462 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 993
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 24
[LightGBM] [Info] Start training from score -0.805476
[LightGBM] [Info] Start training from score -1.563628
[LightGBM] [Info] Start trai

In [160]:
cv_results

,0,1,2,3,4,Mean Accuracy,Std Dev
RandomForest,0.6565,0.6610,0.6555,0.6605,0.6375,0.6542,0.008623
GradientBoost,0.6425,0.6500,0.6620,0.6565,0.6400,0.6502,0.008274
XGBoost,0.6625,0.6705,0.6700,0.6775,0.6560,0.6673,0.007380
LightGBM,0.6645,0.6700,0.6890,0.6810,0.6600,0.6729,0.010679
CatBoost,0.6635,0.6735,0.6740,0.6620,0.6480,0.6642,0.009490
